In [1]:
import json
from pymongo import MongoClient
from googletrans import Translator
from bson.objectid import ObjectId
import tqdm
import re
from nltk import ngrams
import Levenshtein
import pickle
import os
from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec, Doc2Vec
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
f = open('ontology_linked_in_general.json', 'r')
g_onto = json.loads(f.read())

def get_all_skill_from_onto(d_in, skill, exception, level):
    for k in d_in:
        if type(d_in[k]) == dict:
            level+=1
            skill = get_all_skill_from_onto(d_in[k], skill, exception, level)
            level-=1

            if k not in exception and level >=3:
                skill.add(k)
                
    return skill
len(get_all_skill_from_onto(g_onto, set(), ['database', ''], 0))

12666

In [11]:
def cnt(dict_, cnt_, level, cnt_level_1, cnt_level_2, cnt_level_3, cnt_level_4, cnt_level_5):
    for i in dict_:
        if type(dict_[i]) == dict:
            level+=1
            cnt_, level, cnt_level_1, cnt_level_2, cnt_level_3, cnt_level_4, cnt_level_5 = cnt(dict_[i], cnt_, level, cnt_level_1, cnt_level_2, cnt_level_3, cnt_level_4, cnt_level_5)
            cnt_+=1
            if level==1:
                cnt_level_1+=1
            elif level==2:
                cnt_level_2+=1
            elif level==3:
                cnt_level_3+=1
            elif level==4:
                cnt_level_4+=1
            elif level==5:
                cnt_level_5+=1
            level-=1
    return cnt_, level, cnt_level_1, cnt_level_2, cnt_level_3, cnt_level_4, cnt_level_5

cnt(g_onto['all']['linked_in'], 0, 0, 0, 0, 0, 0, 0)

(7052, 0, 6, 68, 571, 6407, 0)

In [27]:
# cnt = []
# cnt_1 = []
# for i in g_onto['all']['linked_in']:
#     for j in g_onto['all']['linked_in'][i]:
#         for k in g_onto['all']['linked_in'][i][j]:
#             if k == 'Software_of_'+j:
#                 cnt_1.append(len(g_onto['all']['linked_in'][i][j][k]))
#             cnt.append(len(g_onto['all']['linked_in'][i][j][k]))
# import numpy as np
# np.sum(cnt)-np.sum(cnt_1)


In [9]:
f = open('ontology_linked_in_general.json', 'r')
g_onto = json.loads(f.read())

def fix_g_onto(d_in, new_onto, exception, level):
    for k in d_in:
        if type(d_in[k]) == dict:
            level+=1
            new_onto = fix_g_onto(d_in[k], new_onto, exception, level)
            level-=1
            
            temp=[]
            for i in d_in[k].keys():
                if i in exception and level >=3:
                    temp.append(i)
            
            for i in temp:
                d_in[k].pop(i, None)
                
    return d_in
new_onto = fix_g_onto(g_onto, {}, ['smartphones', 'smart phone'], 0)

In [39]:
f = open('ontology_linked_in_general.json', 'r')
g_onto = json.loads(f.read())

def fix_g_onto(d_in, new_onto, exception, level):
    for k in d_in:
        if type(d_in[k]) == dict:
            level+=1
            fix_g_onto(d_in[k], new_onto, exception, level)
            level-=1
            
            temp=[]
            for i in d_in[k].keys():
                if i in exception and level >=3:
                    print(i)
                
    return
fix_g_onto(new_onto, {}, ['smartphones', 'smart phone'], 0)

In [40]:
client = MongoClient(host='103.252.1.144', username='admin', password = 'CIST2o20')
cb = client['human_resources']['career_builder']
cv = cb.find({'eng_experience':{'$exists':True}, 'fulltext':{'$regex':'.*developer.*'}})
jpt = client['human_resources']['job_posting_topcv']
jd = jpt.find({'eng_job_description':{'$regex':'.*developer.*'}})

In [1]:
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance
damerau_levenshtein_distance('smtih', 'smith')  # expected result: 1
normalized_damerau_levenshtein_distance('smtih', 'smith')  # expected result: 0.2
damerau_levenshtein_distance([1, 2, 3, 4, 5, 6], [7, 8, 9, 7, 10, 11, 4])  # expected result: 7

from pyxdameraulevenshtein import damerau_levenshtein_distance_seqs, normalized_damerau_levenshtein_distance_seqs
array = ['test1', 'test12', 'test123']
damerau_levenshtein_distance_seqs('test', array)  # expected result: [1, 2, 3]
normalized_damerau_levenshtein_distance_seqs('test', array)  # expected result: [0.2, 0.33333334, 0.42857143]

ModuleNotFoundError: No module named 'pyxdameraulevenshtein'

In [41]:
class CSO_classifier():
    def __init__(self, path='ontology.json'):
        f = open(path, 'r')
        self.g_onto = json.loads(f.read())
        self.g_onto = self.fix_g_onto(g_onto, {}, ['smartphones', 'smart phone'], 0)
        self.skill = self.get_all_skill_from_onto(self.g_onto, set(),['','database', 'knowledge', 'research', 'drawing', 'engineer', 'desktop'], 0)


    def get_all_skill_from_onto(self, d_in, skill, exception, level):
        for k in d_in:
            if type(d_in[k]) == dict:
                level+=1
                skill = self.get_all_skill_from_onto(d_in[k], skill, exception, level)
                level-=1
                if k not in exception and level >=3:
                    skill.add(k)
        return skill

    def fix_g_onto(self, d_in, new_onto, exception, level):
        for k in d_in:
            if type(d_in[k]) == dict:
                level+=1
                new_onto = self.fix_g_onto(d_in[k], new_onto, exception, level)
                level-=1
                
                temp=[]
                for i in d_in[k].keys():
                    if i in exception and level >=3:
                        temp.append(i)
                
                for i in temp:
                    d_in[k].pop(i, None)
                    
        return d_in

    def get_skill_by_concept_similarity(self, sentence):
        sentence = re.sub(r"[-()\"/@;:<>{}`=~|.!?,]", " ", sentence).lower()

        uni_gram = ngrams(sentence.split(), 1)
        bi_gram = ngrams(sentence.split(), 2)
        tri_gram = ngrams(sentence.split(), 3)

        uni_gram = [i[0] for i in list(uni_gram)]
        bi_gram = ['{} {}'.format(i[0],i[1]) for i in list(bi_gram)]
        tri_gram = ['{} {} {}'.format(i[0],i[1], i[2]) for i in list(tri_gram)]
        
        n_gram = uni_gram + bi_gram + tri_gram

        skill_find_out = set()
        
        for i in n_gram:
            for o_k in self.skill:
                if Levenshtein.ratio(i, o_k) > 0.94:
                    skill_find_out.add(o_k)

        return skill_find_out

    
    def build_graph(self, onto, skill, level, skill_same_level, graph):
        for i in onto:
            level+=1
            if type(onto[i]) == dict:
                level, skill_same_level, graph = self.build_graph(onto[i], skill, level, skill_same_level, graph)
    
                pop_ = []
                for j in graph.keys():
                    if j in onto[i].keys():
                        pop_.append(j)
                for j in pop_:
                    if j!= '':
                        tmp = graph[j]
                        graph.pop(j, None)
                        if i not in graph:
                            graph[i]={}
                        graph[i][j] = tmp

                if i in skill:
                    if level not in skill_same_level:
                        skill_same_level[level]=set()
                    skill_same_level[level].add(i)

                if level+1 in skill_same_level:
                    if len(skill_same_level[level+1]) > 0:
                        way_1 = False
                        if i not in graph:
                            graph[i] = list(skill_same_level[level+1])
                        else:
                            for j in skill_same_level[level+1]:
                                if j in graph[i]:
                                    way_1 = True
                            if not way_1 and len(graph[i])>0 and type(graph[i])==dict:
                                for k in skill_same_level[level+1]:
                                    graph[i][k]=[k]
                        
                        skill_same_level[level+1]=set()
                
                level-=1
        return level, skill_same_level, graph
    
    def count_skill_from_graph(self, graph, skill, level, new_graph, cnt=None):
        if(type(graph) == dict):
            for i in graph.keys():
                level+=1
                if cnt!=None:
                    if level==4:
                        cnt.append({i:0})
                
                if type(graph[i]) == dict:
                    skill, cnt, level, new_graph = self.count_skill_from_graph(graph[i], skill, level, new_graph, cnt) 
                    level-=1  
                else:
                    for j in graph[i]:
                        skill.add(j)
                        if cnt!=None and len(cnt)>0:
                            cnt[-1][list(cnt[-1].keys())[0]] += 1
                    level-=1            

        return skill, cnt, level, new_graph

    def get_skill_from_graph(self, graph, skill, level, new_graph, cnt=None):
        if type(graph) == dict:
            for i in graph.keys():
                level+=1
                if cnt!=None:
                    if level==4:
                        cnt.append({i:0})
                
                if type(graph[i]) == dict:
                    skill, cnt, level, new_graph = self.get_skill_from_graph(graph[i], skill, level, new_graph, cnt) 
                    level-=1  
                else:
                    for j in graph[i]:
                        skill.add(j)
                        if cnt!=None and len(cnt)>0:
                            cnt[-1][list(cnt[-1].keys())[0]] += 1
                    level-=1            

            return skill, cnt, level, new_graph
        else:
            return graph

    def build_skill_graph(self, all_skill):
        new_graph = {}
        graph =  self.build_graph(g_onto, all_skill, 0, {}, {})[2]
        count_domain = self.count_skill_from_graph(graph, set(), 0, {}, [])[1]
        not_cancel = True
        
        while len(all_skill)>0 and not_cancel:
            
            max_domain = {'domain': '', 'count': 0}
            for i in count_domain:
                if i[list(i.keys())[0]] > max_domain['count']:
                    max_domain['domain'] = list(i.keys())[0]
                    max_domain['count'] = i[list(i.keys())[0]]
            
            for i in graph:
                for k in graph[i]:
                    for l in graph[i][k]:
                        for j in graph[i][k][l]:
                            if j == max_domain['domain']:
                                sub_skill = self.get_skill_from_graph(graph[i][k][l][j], set(), 0, {})[0]
                                if i not in new_graph:
                                    new_graph[i] = {}
                                if k not in new_graph[i]:
                                    new_graph[i][k] = {}
                                if l not in new_graph[i][k]:
                                    new_graph[i][k][l] = {}
                                if j not in new_graph[i][k][l]:
                                    new_graph[i][k][l][j] = {}
                                
                                new_graph[i][k][l][j] = graph[i][k][l][j]
            
            if type(sub_skill) == str:
                all_skill=all_skill-set([sub_skill])
            else:
                all_skill=all_skill-sub_skill

            graph = self.build_graph(g_onto, all_skill, 0, {}, {})[2]

            count_domain = self.count_skill_from_graph(graph, set(), 0, {}, [])[1]
            
            if len(count_domain)==0:
                not_cancel = False
        return new_graph

                                                                                                                    

In [42]:
ontology = {'A':[], 'JD': [], 'R': []}

c = {'Profile': None, 'Work_history': None, 'Education': None, 'Course': None, 'Skills': {}, 'Pos': None, 'Others': None}
d = {'Information': None, 'Descriptions': None, 'Required': None, 'Skills': {}}

cso_classifier = CSO_classifier()

In [43]:
skills = cso_classifier.get_skill_by_concept_similarity(' '.join(cv[0]['eng_experience']+cv[0]['eng_skills']))


In [44]:
cso_classifier.build_skill_graph(skills)

{'all': {'general': {'computer science': {'software engineering': {'software design': {'software architecture': {'software development': ['software team'],
       'software development process': ['software team'],
       'software development life cycle': ['software team'],
       'software development project': ['software team'],
       'software developer': ['software developer']},
      'software product': {'software development': ['software team'],
       'software development process': ['software team'],
       'software development life cycle': ['software team'],
       'software development project': ['software team']},
      'software project': {'software development': ['software team'],
       'software development process': ['software team'],
       'software development life cycle': ['software team'],
       'software development project': ['software team']},
      'software process': {'software development': ['software team'],
       'software development process': ['softwa

In [45]:
print(cso_classifier.build_graph(g_onto,skills, 0, {},{})[2])

{'all': {'general': {'computer science': {'artificial intelligence': {'machine learning': {'pattern recognition': {'face recognition': ['face detection']}}, 'programming language': ['python', 'c++']}, 'data science': {'programming language': ['python']}, 'computer network': {'wireless telecommunication system': {'communication channel %28information theory%29': {'mobile telecommunication system': {'smartphone': ['android']}}}}, 'software engineering': {'software design': {'software architecture': {'software development': ['software team'], 'software development process': ['software team'], 'software development life cycle': ['software team'], 'software development project': ['software team'], 'software developer': ['software developer']}, 'software product': {'software development': ['software team'], 'software development process': ['software team'], 'software development life cycle': ['software team'], 'software development project': ['software team']}, 'software project': {'software